In [1]:
include("./src/parse_file.jl")

process_file (generic function with 1 method)

In [2]:
path = "./testing/test_epiread_small.bed"
big_bgz_path = "/Volumes/projects/laird/nathan/projects/julia/epiread/testing/test_epiread.bed.bgz"
part_path = "/Volumes/projects/laird/nathan/projects/julia/epiread/testing/part_epiread.bed"


"/Volumes/projects/laird/nathan/projects/julia/epiread/testing/part_epiread.bed"

In [3]:
results = parse_infile(path)

Parsed 63 reads in chr1
Completed analyis of 37 reads in chr2


Dict{String, SparseMatrixCSC{Int64, Int64}} with 2 entries:
  "chr2" => sparse([2, 2, 4, 6, 2, 4, 6, 4, 6, 4  …  15, 15, 16, 16, 17, 18, 17…
  "chr1" => sparse([2, 2, 2, 2, 2, 3, 3, 1, 1, 4  …  23, 24, 25, 26, 27, 21, 22…

In [4]:
x = results["chr1"]

27×631931 SparseMatrixCSC{Int64, Int64} with 105 stored entries:
⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠈⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠒⠂⠐

In [5]:
function tally_epialles(matrix::SparseMatrixCSC{Int64, Int64}, chr::String; window_size = 4::Int)
    n_bases = size(matrix, 2)
    current_bed_line = (0::Int, 0::Int) # (start, n_alleles)
    for i in 1:window_size:n_bases
        if i % 100000 == 1
            println(Base.stderr, "Tallying epialleles on $(chr) at position $(i) of $(n_bases)")
            flush(stdout)
        end
        upper_limit = minimum([i+window_size-1, n_bases])
        a = get_unsparse_nonzero_rows(matrix[:,i:upper_limit])

        n = 0
        if !isempty(a)
            n = count_epialleles(matrix_to_epiallele_vector(a))
        end

        if n != current_bed_line[2]
            # println("$(chr)\t$(current_bed_line[1])\t$(i-1)\t$(current_bed_line[2])")
            current_bed_line = (i-1, n)
        end
    end
end

tally_epialles (generic function with 1 method)

In [16]:
@profview tally_epialles(x, "chr1")

Tallying epialleles on chr1 at position 1 of 631931
Tallying epialleles on chr1 at position 100001 of 631931
Tallying epialleles on chr1 at position 200001 of 631931
Tallying epialleles on chr1 at position 300001 of 631931
Tallying epialleles on chr1 at position 400001 of 631931
Tallying epialleles on chr1 at position 500001 of 631931
Tallying epialleles on chr1 at position 600001 of 631931


UndefVarError: UndefVarError: vscode not defined

In [ ]:
function get_unsparse_nonzero_rows(matrix::SparseMatrixCSC{Int64, Int64})
	nzrows = unique(matrix.rowval)
	if !isempty(nzrows)
		return Matrix(matrix)[nzrows,:]
	else 
		return zeros(Int64, (0,0))
	end
end

In [21]:
@code_warntype get_unsparse_nonzero_rows(x[:,1:4])

MethodInstance for get_unsparse_nonzero_rows(::SparseMatrixCSC{Int64, Int64})
  from get_unsparse_nonzero_rows(matrix::SparseMatrixCSC{Int64, Int64}) in Main at /Volumes/projects/laird/nathan/projects/julia/epiread/src/parse_file.jl:335
Arguments
  #self#::Core.Const(get_unsparse_nonzero_rows)
  matrix::SparseMatrixCSC{Int64, Int64}
Locals
  nzrows::Vector{Int64}
Body::Matrix{Int64}
1 ─ %1  = Base.getproperty(matrix, :rowval)::Vector{Int64}
│         (nzrows = Main.unique(%1))
│   %3  = Main.isempty(nzrows)::Bool
│   %4  = !%3::Bool
└──       goto #3 if not %4
2 ─ %6  = Main.Matrix(matrix)::Matrix{Int64}
│   %7  = nzrows::Vector{Int64}
│   %8  = Base.getindex(%6, %7, Main.:(:))::Matrix{Int64}
└──       return %8
3 ─ %10 = Core.tuple(0, 0)::Core.Const((0, 0))
│   %11 = Main.zeros(Main.Int64, %10)::Matrix{Int64}
└──       return %11



In [22]:
@profview get_unsparse_nonzero_rows(x[:,1:4])

┌ Warning: There were no samples collected.
│ Run your program longer (perhaps by running it multiple times),
│ or adjust the delay between samples with `Profile.init()`.
└ @ Profile /Users/administrator/.julia/scratchspaces/a66863c6-20e8-4ff4-8a62-49f30b1f605e/agent-cache/default-macmini-aarch64-1.0/build/default-macmini-aarch64-1-0/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Profile/src/Profile.jl:1208
